In [ ]:
import pandas as pd
import duckdb
import argparse
import yaml
import os

In [ ]:
conn = duckdb.connect()

In [ ]:
# Specify the filename
filename9 = './data/input/Section111ValidICD9-Jan2024.csv'
filename10 = './data/input/Section111ValidICD10-Jan2024_edit.csv'

In [ ]:

# Create a DuckDB table from the CSV file
conn.execute(f'CREATE TABLE valid_icd_9 AS SELECT * FROM read_csv_auto(\'{filename9}\', ALL_VARCHAR=1)')


In [ ]:

# Create a DuckDB table from the CSV file
conn.execute(f'CREATE TABLE valid_icd_10 AS SELECT * FROM read_csv_auto(\'{filename10}\', ALL_VARCHAR=1)')

In [ ]:

# Execute the SQL query and fetch the results
query = 'SELECT * FROM valid_icd_9 LIMIT 5'
result = conn.execute(query).fetchdf()


In [ ]:
result

In [ ]:

# Execute the SQL query and fetch the results
query2 = 'SELECT * FROM valid_icd_10 LIMIT 5'
result2 = conn.execute(query2).fetchdf()
result2

In [ ]:

# Execute the SQL query and fetch the results
query = """
SELECT *
FROM valid_icd_10
WHERE column0 LIKE 'G20%'
   OR column0 LIKE 'G2111%'
ORDER BY column0"""
result = conn.execute(query).fetchdf()
result.column0

In [ ]:
SELECT 
        * 
    FROM 
        read_csv_auto('icd10_valid.csv', header = TRUE)
    WHERE 
        SUBSTRING(icd10, 1, 3) >= 'I00' AND SUBSTRING(icd10, 1, 3) <= 'I99' OR
        SUBSTRING(icd10, 1, 4) >= 'Q200' AND SUBSTRING(icd10, 1, 4) <= 'Q280'


In [ ]:

# Define the path to the YAML file
yaml_file_path = './conf/icd_codes/icd_codes_2.yml'

# Read YAML file into a dictionary
with open(yaml_file_path, 'r') as yaml_file:
    icd_codes = yaml.safe_load(yaml_file)


In [ ]:

# Initialize empty dictionaries to store results
icd9_result_dict = {}
icd10_result_dict = {}

# Loop through each section in icd_codes dictionary
for key, values in icd_codes.items():
    icd9_result_list = []
    icd10_result_list = []

    for icd9_code in values["icd9"]:
        query = f"""
        SELECT *
        FROM valid_icd_9
        WHERE CODE LIKE '{icd9_code}%'
        ORDER BY CODE
        """
        result = conn.execute(query).fetchdf()
        icd9_result_list.extend(result.CODE.tolist())

    for icd10_code in values["icd10"]:
        query = f"""
        SELECT *
        FROM valid_icd_10
        WHERE column0 LIKE '{icd10_code}%'
        ORDER BY column0
        """
        result = conn.execute(query).fetchdf()
        icd10_result_list.extend(result.column0.tolist())

    # Add icd9 and icd10 result lists to respective dictionaries
    icd9_result_dict[key] = {"icd9": icd9_result_list}
    icd10_result_dict[key] = {"icd10": icd10_result_list}


In [ ]:
icd9_result_dict

In [ ]:
icd10_result_dict

In [ ]:
%%bash
python get_outcomes.py --year 2015


In [ ]:
query = """SELECT bene_id, adm_id 
FROM (SELECT bene_id, adm_id, UNNEST(diagnoses) AS diag FROM './data/input/mbsf_medpar_denom/medpar_hospitalizations_2015.parquet')
WHERE diag IN ('3320','3321','G20','G20A1','G20A2','G20B1','G20B2','G20C','G2111','G2119','G213','G214','G218','G219')
"""

In [ ]:
outcome_df = conn.execute(query).fetchdf()

In [1]:
%%bash

for year in {2000..2016}
do
    python get_outcomes.py --year $year
done

Traceback (most recent call last):
  File "/n/dominici_nsaph_l3/Lab/data_processing/medpar_outcomes/kezia/get_outcomes.py", line 1, in <module>
    import pandas as pd
  File "/n/home_fasse/kirene/.conda/envs/dorieh/lib/python3.10/site-packages/pandas/__init__.py", line 48, in <module>
    from pandas.core.api import (
  File "/n/home_fasse/kirene/.conda/envs/dorieh/lib/python3.10/site-packages/pandas/core/api.py", line 27, in <module>
    from pandas.core.arrays import Categorical
  File "/n/home_fasse/kirene/.conda/envs/dorieh/lib/python3.10/site-packages/pandas/core/arrays/__init__.py", line 1, in <module>
    from pandas.core.arrays.arrow import ArrowExtensionArray
  File "/n/home_fasse/kirene/.conda/envs/dorieh/lib/python3.10/site-packages/pandas/core/arrays/arrow/__init__.py", line 1, in <module>
    from pandas.core.arrays.arrow.array import ArrowExtensionArray
  File "/n/home_fasse/kirene/.conda/envs/dorieh/lib/python3.10/site-packages/pandas/core/arrays/arrow/array.py", line 5

Error while terminating subprocess (pid=2891803): 


In [1]:

import pyarrow.parquet as pq
import pandas as pd


In [2]:

# Specify the path to your Parquet file
parquet_file = 'data/output/medpar_outcomes/icd_codes_4/all_icd_2000.parquet'

# Read the Parquet file into a Pandas DataFrame
table = pq.read_
table(parquet_file)
df = table.to_pandas()



NameError: name 'pq' is not defined